http://blog.csdn.net/weixin_35653315/article/details/71015845 中描述了将pascal voc数据集转换tfrecord的过程。本文则通过slim来读取生成的tfrecord。读取和解码操作由`tf.TFRecordReader`完成

In [1]:
import tensorflow as tf
slim = tf.contrib.slim
file_pattern = './pascal_train_*.tfrecord' #文件名格式

# 适配器1：将example反序列化成存储之前的格式。由tf完成
keys_to_features = {
    'image/encoded': tf.FixedLenFeature((), tf.string, default_value=''),
    'image/format': tf.FixedLenFeature((), tf.string, default_value='jpeg'),
    'image/height': tf.FixedLenFeature([1], tf.int64),
    'image/width': tf.FixedLenFeature([1], tf.int64),
    'image/channels': tf.FixedLenFeature([1], tf.int64),
    'image/shape': tf.FixedLenFeature([3], tf.int64),
    'image/object/bbox/xmin': tf.VarLenFeature(dtype=tf.float32),
    'image/object/bbox/ymin': tf.VarLenFeature(dtype=tf.float32),
    'image/object/bbox/xmax': tf.VarLenFeature(dtype=tf.float32),
    'image/object/bbox/ymax': tf.VarLenFeature(dtype=tf.float32),
    'image/object/bbox/label': tf.VarLenFeature(dtype=tf.int64),
    'image/object/bbox/difficult': tf.VarLenFeature(dtype=tf.int64),
    'image/object/bbox/truncated': tf.VarLenFeature(dtype=tf.int64),
}

#适配器2：将反序列化的数据组装成更高级的格式。由slim完成
items_to_handlers = {
    'image': slim.tfexample_decoder.Image('image/encoded', 'image/format'),
    'shape': slim.tfexample_decoder.Tensor('image/shape'),
    'object/bbox': slim.tfexample_decoder.BoundingBox(
            ['ymin', 'xmin', 'ymax', 'xmax'], 'image/object/bbox/'),
    'object/label': slim.tfexample_decoder.Tensor('image/object/bbox/label'),
    'object/difficult': slim.tfexample_decoder.Tensor('image/object/bbox/difficult'),
    'object/truncated': slim.tfexample_decoder.Tensor('image/object/bbox/truncated'),
}

# 解码器
decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)

# dataset对象负责数据的读取和解码
dataset = slim.dataset.Dataset(
            data_sources=file_pattern,
            reader=tf.TFRecordReader,
            num_samples = 3, # 手动生成了三个文件， 每个文件里只包含一个example
            decoder=decoder,
            items_to_descriptions = {},
            num_classes=21)

#provider对象负责多线程
provider = slim.dataset_data_provider.DatasetDataProvider(
                    dataset,
                    num_readers=3,
                    common_queue_capacity=20,
                    common_queue_min=10,
                    shuffle=True)

[image, shape, glabels, gbboxes] = provider.get(['image', 'shape',
                                                             'object/label',
                                                             'object/bbox'])
print type(image)
print image.shape

<class 'tensorflow.python.framework.ops.Tensor'>
(?, ?, 3)


到目前为止，返回的image是一个tensor，而且是一个3维的，一次只有一张。先来看看读取出来的图片。

In [ ]:
with tf.Session() as sess:
    img = sess.run([image])
    print img.shape